# Using Arcpy to Automate GIS Tasks

## Introduction

Below is an example of using Python and the Arcpy library to download Digital Elevation Models (DEMs) from the USGS website using their API. This is done by Hydrologic Unit Code (HUC) 8 in this example. Where 8 is the number of digits in the HUC code, the more numbers the smaller the HUC area, the smallest being HUC12.

Once the DEMs in the desired area are downloaded, ArcPy is used to mosaik the DEMs into a single raster, and then clips them using the HUC 8 boundary. 

## Importing Libraries

In order to use arcpy, one must have ArcGIS installed on their computer, and have recently logged into an ArcGIS account. This is because the ArcPy library is a part of the ArcGIS software.

In [ ]:
import json
import requests
import pandas as pd
import os
import arcpy

## Downloading DEMs

### Set Parameters and Make Request

Here we setup the parameters for our workspace, including the HUC 8 code, the output directory, and the output file name as well as the basis for the USGS API URL. We also make our initial request to the USGS API to get the list of DEMs in the HUC 8 area.

In [ ]:
s_url = 'https://tnmaccess.nationalmap.gov/api/v1/products?datasets='
dataset = 'National Elevation Dataset (NED) 1/3 arc-second'
polyCode = input('Enter HUC8: ')
polyType = 'huc8'

url = s_url + dataset + '&polyCode=' + polyCode + '&polyType=' + polyType
sp = 'dem\\'+ polyCode

os.makedirs(sp, exist_ok=True)

r = requests.get(url)
data = r.json()

### Download the DEMs

This next portion of code is used to download the DEMs from the USGS API. First we create an output directory, then We iterate through the list of DEMs and download them to it.

In [ ]:
for i in data.get('items'):

    url = i.get('downloadURL')
    fname = url.split('/')[-1]
    spath = sp + '/' + fname

    if os.path.exists(spath):
        print('Already exists: ' + fname)
        continue
    
    else:
        print('Downloading: ' + fname)
        r = requests.get(url)
        open(spath, 'wb').write(r.content)

## HUC 8 Boundary

Here we are retriving the HUC 8 boundary from our workspace directory. This will be used to clip the DEMs.

In [ ]:
wdb_path = sp + '\\' + polyCode + '_wbd.shp'

wd = os.getcwd()
wd += '\\' + 'wbd'

arcpy.env.workspace = wd
arcpy.env.overwriteOutput = True

if os.path.exists(wdb_path):
    print('Already exists: ' + wdb_path)

else:
    print(f'Exporting WDB HUC8 for {polyCode}')
    huc8 = f"huc8 = '{polyCode}'"
    arcpy.conversion.ExportFeatures("CA_WBD_HU8", wdb_path,
        huc8,
        "NOT_USE_ALIAS",
        'tnmid "tnmid" true true false 40 Text 0 0,First,#,CA_WBD_HU8,tnmid,0,40;metasource "metasource" true true false 40 Text 0 0,First,#,CA_WBD_HU8,metasource,0,40;sourcedata "sourcedata" true true false 100 Text 0 0,First,#,CA_WBD_HU8,sourcedata,0,100;sourceorig "sourceorig" true true false 130 Text 0 0,First,#,CA_WBD_HU8,sourceorig,0,130;sourcefeat "sourcefeat" true true false 40 Text 0 0,First,#,CA_WBD_HU8,sourcefeat,0,40;loaddate "loaddate" true true false 8 Date 0 0,First,#,CA_WBD_HU8,loaddate,-1,-1;referenceg "referenceg" true true false 50 Text 0 0,First,#,CA_WBD_HU8,referenceg,0,50;areaacres "areaacres" true true false 19 Double 0 0,First,#,CA_WBD_HU8,areaacres,-1,-1;areasqkm "areasqkm" true true false 19 Double 0 0,First,#,CA_WBD_HU8,areasqkm,-1,-1;states "states" true true false 50 Text 0 0,First,#,CA_WBD_HU8,states,0,50;huc8 "huc8" true true false 8 Text 0 0,First,#,CA_WBD_HU8,huc8,0,8;name "name" true true false 120 Text 0 0,First,#,CA_WBD_HU8,name,0,120;shape_Leng "shape_Leng" true true false 19 Double 0 0,First,#,CA_WBD_HU8,shape_Leng,-1,-1;ObjectID "ObjectID" true true false 10 Long 0 10,First,#,CA_WBD_HU8,ObjectID,-1,-1;Shape_Le_1 "Shape_Le_1" true true false 19 Double 0 0,First,#,CA_WBD_HU8,Shape_Le_1,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,CA_WBD_HU8,Shape_Area,-1,-1',
        None)

## Mosaic

In the following cell, we are using ArcPy to gather the list of DEMs in our workspace directory and then create a mosaic of them all. While in many modern GIS this step may not be neccesary as rasters can be displayed and processed on the fly, for this script's application in ArcMap this is not the case.

In [ ]:
wd = os.getcwd()
wd += "\\" + sp

arcpy.env.workspace = wd
arcpy.env.overwriteOutput = True

rasters = arcpy.ListRasters()

fname = polyCode + "_mosaic.tif"
fpath = wd + '\\' + fname

if os.path.exists(fname):
    print('Already exists: ' + fname)

else:
    print('Mosaicking: ' + fname)
    with arcpy.EnvManager(resamplingMethod="BILINEAR", pyramid="PYRAMIDS -1 BILINEAR DEFAULT 75 NO_SKIP NO_SIPS"):
        arcpy.management.MosaicToNewRaster(rasters, wd, fname, None, "32_BIT_FLOAT", None, 1, "MEAN", "FIRST")

## Clip

Now that we have the HUC 8 boundary and the mosaiked DEM, we can clip the DEM to the boundary.

In [ ]:
print('Clipping: ' + fname)
with arcpy.da.SearchCursor(arcpy.ListFeatureClasses()[0], ['Shape@']) as rows:
    for row in rows:
        extent = row[0].extent.XMin, row[0].extent.YMin, row[0].extent.XMax, row[0].extent.YMax
        extent = str(extent).replace('(', '').replace(')', '').replace(',', '')

cpath = wd + '\\' + polyCode + '_mclip.tif'

with arcpy.EnvManager(resamplingMethod="BILINEAR", pyramid="PYRAMIDS -1 BILINEAR DEFAULT 75 NO_SKIP NO_SIPS"):
    arcpy.management.Clip(fname, extent, cpath, wdb_path, "3.4e+38", "ClippingGeometry", "NO_MAINTAIN_EXTENT")

print('Completed: ' + cpath)

## Conclusion

In this notebook, we used Python and ArcPy to download DEMs from the USGS website using their API. We then mosaiked the DEMs into a single raster, and clipped them to the HUC 8 boundary. The purpose of this was to create a DEM for a specific area that can be used in hydrologic modeling. In practice it was used to more quickly setup a GIS environment for digitizing hydrologic features.

Below is an example of the output of this notebook:

In [ ]:
# Place holder cell for clipped DEM output